### **⬇ Generar Texto_entrada.txt ⬇**

In [ ]:
def generar_archivo_grande(ruta, lineas=1000000):
    """Genera un archivo de texto grande para pruebas de rendimiento."""
    with open(ruta, 'w') as f:
        for i in range(lineas):
            f.write("    esto es una linea con espacios.    \n")
            f.write("otra linea en minusculas\n")
            f.write("  Y OTRA CON ESPACIOS Y MAYUSCULAS  \n")
            f.write("   linea adicional para procesar   \n")
            f.write("mas lineas para tener suficientes datos\n")  # Con espacios al inicio y al final

if __name__ == '__main__':
    ruta_entrada = "texto_entrada.txt"
    generar_archivo_grande(ruta_entrada)
    print(f"Archivo grande '{ruta_entrada}' generado con 5.000.000 líneas.")


Archivo grande 'texto_entrada.txt' generado con 5.000.000 líneas.


### **⬇ Algoritmo Secuencial ⬇**

In [ ]:
import time

def procesar_texto_secuencial(ruta_entrada, ruta_salida):
    """Procesa el archivo de texto secuencialmente."""
    try:
        with open(ruta_entrada, 'r') as f_in, open(ruta_salida, 'w') as f_out:
            for linea in f_in:
                linea_limpia = linea.strip()
                linea_mayusculas = linea_limpia.upper()
                f_out.write(linea_mayusculas + '\n')
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {ruta_entrada}")

if __name__ == '__main__':
    ruta_entrada = "texto_entrada.txt"
    ruta_salida = "texto_salida_secuencial.txt"

    inicio = time.time()
    procesar_texto_secuencial(ruta_entrada, ruta_salida)
    fin = time.time()

    print(f"Tiempo total de procesamiento secuencial: {fin - inicio:.2f} segundos")
    print(f"Archivo procesado secuencialmente guardado en {ruta_salida}")


Tiempo total de procesamiento secuencial: 3.55 segundos
Archivo procesado secuencialmente guardado en texto_salida_secuencial.txt


### **⬇ Algoritmo Pipeline (con descomposición funcional) ⬇**

In [ ]:
import multiprocessing
import time

# Tamaño del bloque en bytes
TAM_BLOQUE = 1024 * 1024  # 1 MB

def leer_datos(archivo_entrada, cola):
    """Lee el archivo en bloques de bytes y los envía a la cola."""
    with open(archivo_entrada, "rb") as f:
        while True:
            bloque = f.read(TAM_BLOQUE)
            if not bloque:
                break
            cola.put(bloque)
    cola.put(None)  # Señal de fin

def limpiar_datos(cola_entrada, cola_salida):
    """Elimina espacios solo al inicio y al final, pero mantiene los internos."""
    while True:
        bloque = cola_entrada.get()
        if bloque is None:
            cola_salida.put(None)
            break
        # Se convierte bytes a lineas, se limpian los espacion en los extremos y se reconstruye el bloque
        lineas = bloque.decode("utf-8").split("\n")  # Se divide en líneas
        lineas_limpias = [linea.strip() for linea in lineas]  # Se eliminan lo espacios extremos
        bloque_limpio = "\n".join(lineas_limpias).encode("utf-8")  # Reconvierte a bytes
        cola_salida.put(bloque_limpio)

def transformar_datos(cola_entrada, cola_salida):
    """Convierte los bloques de bytes a mayúsculas."""
    while True:
        bloque = cola_entrada.get()
        if bloque is None:
            cola_salida.put(None)
            break
        bloque_transformado = bloque.upper()
        cola_salida.put(bloque_transformado)

def escribir_datos(cola_entrada, archivo_salida):
    """Escribe los bloques de bytes transformados al archivo de salida."""
    with open(archivo_salida, "wb") as f:
        while True:
            bloque = cola_entrada.get()
            if bloque is None:
                break
            f.write(bloque)

def procesar_texto_pipeline(entrada, salida):
    """Procesa el archivo de texto usando un pipeline paralelo."""
    cola_lectura = multiprocessing.SimpleQueue()
    cola_limpieza = multiprocessing.SimpleQueue()
    cola_transformacion = multiprocessing.SimpleQueue()

    procesos = [
        multiprocessing.Process(target=leer_datos, args=(entrada, cola_lectura)),
        multiprocessing.Process(target=limpiar_datos, args=(cola_lectura, cola_limpieza)),
        multiprocessing.Process(target=transformar_datos, args=(cola_limpieza, cola_transformacion)),
        multiprocessing.Process(target=escribir_datos, args=(cola_transformacion, salida)),
    ]

    # Iniciar procesos
    for p in procesos:
        p.start()

    # Esperar a que terminen
    for p in procesos:
        p.join()

if __name__ == "__main__":
    ruta_entrada = "texto_entrada.txt"
    ruta_salida_pipeline = "texto_salida_pipeline.txt"

    inicio = time.time()
    procesar_texto_pipeline(ruta_entrada, ruta_salida_pipeline)
    fin = time.time()

    print(f"Tiempo total de procesamiento con pipeline en bytes: {fin - inicio:.2f} segundos")
    print(f"Archivo procesado en paralelo guardado en {ruta_salida_pipeline}")


Tiempo total de procesamiento con pipeline en bytes: 2.56 segundos
Archivo procesado en paralelo guardado en texto_salida_pipeline.txt


### **⬇ Comparar Resultados ⬇**

In [ ]:
# Rutas de archivos
ruta_entrada = "texto_entrada.txt"
ruta_salida_secuencial = "texto_salida_secuencial.txt"
ruta_salida_pipeline = "texto_salida_pipeline.txt"

# Ejecutar procesamiento secuencial
print("Ejecutando procesamiento secuencial...")
inicio_sec = time.time()
procesar_texto_secuencial(ruta_entrada, ruta_salida_secuencial)
fin_sec = time.time()
tiempo_sec = fin_sec - inicio_sec

# Ejecutar procesamiento con pipeline
print("\nEjecutando procesamiento con pipeline en bytes...")
inicio_pipe = time.time()
procesar_texto_pipeline(ruta_entrada, ruta_salida_pipeline)
fin_pipe = time.time()
tiempo_pipe = fin_pipe - inicio_pipe

# Comparar tiempos
print("\n**COMPARACIÓN FINAL**")
print(f"Secuencial: {tiempo_sec:.2f} s")
print(f"Pipeline en bytes: {tiempo_pipe:.2f} s")
mejora = ((tiempo_sec - tiempo_pipe) / tiempo_sec) * 100
print(f"**Mejora con pipeline: {mejora:.2f}%**")


🔄 Ejecutando procesamiento secuencial...

🚀 Ejecutando procesamiento con pipeline en bytes...

📊 **COMPARACIÓN FINAL**
✅ Secuencial: 2.07 s
🚀 Pipeline en bytes: 2.56 s
⚡ **Mejora con pipeline: -23.47%**
